# Subisistence Fishing on Cape Fear River

Going through the data from intercept surveys, WIC clinic visits and online surveys collected during fall / winter 2019-2020 as part of the Steven Yang and Martin Dietz's master's project available [here]( https://dukespace.lib.duke.edu/dspace/handle/10161/20557)

## Table of Contents
* [Setup](#Setup)
	* [Useful Functions](#Useful-functions)
    * [Cleaning the Data](#Cleaning-the-Data)
* [Presenting our Results](#Presenting-our-Results)
    * [Demographics](#Demographics)
    * [Fish Species Consumed](#Fish-Species-Consumed)
    * [Fishing Locations](#Fishing-Locations)
    * [Quantity of Fish Consumed](#Quantity-of-Fish-Consumed)
    * [Fish Sharing Behaviors](#Fish-Sharing-Behaviors)
* [More Detailed Results](#More-Detailed-Results)
    * [What do People do with Fish Other Than Eating?](#What-do-People-do-With-Fish-Other-Than-Eating?)
    * [How are People Preparing and Cooking the Fish](#How-are-People-Preparing-and-Cooking-the-Fish?)
    * [Are Fishing Advisories Visible and Easy to Understand?](#Are-Fishing-Advisories-Visible-and-Easy-to-Understand?)
    * [Pollution Concerns](#Pollution-Concerns)
	* [Sources of Information](#Sources-of-Information)
        * [Where do people get information on which fish are safe to eat?](#Where-do-people-get-information-on-which-fish-are-safe-to-eat?)
        * [Where do people get trusted health information?](#Where-do-people-get-trusted-health-information?)
    * [Feedback on Informational Materials](#Feedback-on-Informational-Materials)
        * [Wallet Card](#Wallet-Card)
		* [Refrigerator Magnet](#Refrigerator-Magnet)
    * [Likelihood, barriers, and motivations for different behaviors](#Likelihood,-barriers,-and-motivations-for-different-behaviors)
    * [Final Comments](#Final-Comments)


## Setup
I do a few things here to setup for exploring our dataset:
1. Import the library I used for programming ('pandas')
2. Read in the data set. This data is from a 'master' csv file in which I combined the responses from intercept surveys, online surveys and WIC clinic surveys. 
3. Write a couple of helper functions that will be used throughout. 
4. Clean up the data, keeping only complete responses by filtering out our trial runs through the surveys and the responses from participants who don't eat fish from the river and therefore didn't go through all the questions.

In [2]:
import pandas as pd

In [3]:
# not read_csv has many paramaters you can specify. First tring with no specifications
df = pd.read_csv('master.csv')
df.shape  # tells us that there are 103 rows and 96 columns in this dataset

(103, 95)

#### Useful functions
Now I'll define a couple functions that will be useful for interpreting our data. __Please ignore this if you're not interested in the programming I did on the backend!__  The first function aggregates across multiple choice responses. When people selected many things, like many different cooking or preparation methods, this goes through and aggregates how many times each option was chosen. The second function filters out blanks and then prints a list of free form responses under a title starting with '>>>'

In [4]:
def aggregate_mult_choice(column_header):
    mask = df_completed[column_header].notna()  # This creates a mask we use to filter out blank responses
    resp_list = df_completed[column_header][mask].tolist()
    response_dict = {}
    for resp in resp_list:
        for choice in resp.split(','):
            choice = choice.strip().lower()
            if choice not in response_dict:
                response_dict[choice] = 0
            response_dict[choice] += 1
    return response_dict
def print_freeform_columns(list_of_headers):
    for header in list_of_headers:  # This goes through each column in the list, filters blanks, and prints the responses
            mask = df_completed[header].notna()
            resp_list = df_completed[mask][header].tolist()
            print('>>> ' + header[10:] + ' (verbal responses): ')   # this removes the 'FREE FORM ' from column head and prints the rest/
            for resp in resp_list:
                print(resp)

#### Cleaning the Data
We mostly completed intercept surveys with some activity online and a few from WIC clinics. Still, the above dataframe  ('df_answered') includes some respondents who didn't eat fish and also some tests of the survey that Steven and I completed. A condition for taking the whole survey was that the individual eats fish from the river, so I'll make a dataframe of qualified responses by keeping only the responses in which the respondent said 'Yes' to eating fish from the river, and therefore qualified to take the survey.

In [45]:
df_answered = df[df.finished == True]  # filtering out the responses which qualtrics determined were unfinished
df_answered.shape # lets look at the shape of the new data set

(76, 95)

In [6]:
df_qualifies = df_answered[df_answered['resp_eat_fish'] == 'Yes']  # keeping only responses in which the respondent eats fish from CFR
locationMask = df_qualifies.location.notna()  # making a mask for responses that are not blank ('NaN')
df_completed = df_qualifies[locationMask]  # Now we create the modified database with complete responses 
df_completed.shape  # now we've kept only 45 rows (each representing a response) but still 96 columns (fields)

(45, 95)

To start lets see counts of responses by source (intercept, online, WIC)

In [7]:
df_completed['source'].value_counts()

intercept    37
online        6
WIC           2
Name: source, dtype: int64

So most of our completed surveys were intercept surveys, with a few online and a couple WIC

## Presenting our Results

### Demographics

In the survey we asked people's gender identification, education level, and age. We also observed and recorded race, but those numbers can be seen in our presentation and write-up and are not in this dataset.

In [8]:
df_completed['age'].value_counts().sort_index()

18-20           1
21-29           6
30-39          11
40-49           4
50-59          10
60 or older    12
Name: age, dtype: int64

In [9]:
df_completed['gender'].value_counts()

Male      36
Female     8
Name: gender, dtype: int64

In [10]:
df_completed['educ'].value_counts()

High school diploma, or equivalent (for example, GED)    14
Some college, no degree                                   9
Bachelor degree                                           6
Associate degree                                          6
Less than high school degree                              5
Graduate degree                                           2
Name: educ, dtype: int64

This shows how we got our demographic figures, and it's clear that we reached mostly men of varying ages but of predominantly lower education levels.

### Fish Species Consumed

In [11]:
# This gives me a list of the columns with data on the fish people eat. I'll use it to make another smaller dataframe
col_list = []
for col in df_completed.columns: 
    if col[0:3] == 'eat':
        col_list.append(col)
df_complete_fish = df_completed[col_list]

'On' values means that the respondent selected that fish as a species they consume. I'd like to aggregate how many are 'On' in each column and present that.

In [12]:
# To make it easier to add, I want to replace 'On' with 1 and 'Off' with 0:
df_complete_fish = df_complete_fish.replace({'On': 1, 'Off': 0})  # had to reassign it to a new df to avoid modifying df in place errors

In [13]:
# Now to sum up each column and then present that row
df_complete_fish = df_complete_fish.append(df_complete_fish.sum().rename('Total'))
df_complete_fish.tail(1)

,eats Flathead Catfish,eats Bowfin (Blackfish),eats Blue Catfish,eats Channel Catfish,eats White Catfish,eats American Shad,eats Bluegill,eats Redbreast Sunfish,eats White Crappie,eats Striped Bass,eats Carp,eats Black Crappie,eats Warmouth,eats Green Sunfish,eats Largemouth Bass,eats Redear Sunfish,eats Hickory Shad
Total,8,0,9,10,5,2,4,1,1,7,0,3,2,0,3,3,1


This shows the total! <br> <font color = 'blue' > NOTE: the above totals exclude the write-in responses! Our chart showed mostly freshwater fish, but it turns out the people eat many brackish water species. <br> Also NOTE: we didn't make it clear that we were also asking about shellfish. More people than reported might eat blue crab.<br> Final NOTE: bluegill showed up twice in survey's menu of options (maybe intentional to show two different types?). In cleaning up the data I combined the responses in to one column for bluegill.

Now let's take a look at the verbal responses that Steven and I recorded:

In [14]:
verbal_fish_dict = aggregate_mult_choice('FREE FORM other fish')
sorted(verbal_fish_dict.items(), key = lambda x: x[1], reverse = True)  # This prints the dictionary in order - lots of drum!

[('red drum', 21),
 ('black drum', 19),
 ('speckled trout', 15),
 ('flounder', 8),
 ('croaker', 7),
 ('spot', 6),
 ('sheepshead', 6),
 ('blue fish', 5),
 ('whiting', 4),
 ('pig fish', 2),
 ('blue crab', 2),
 ('chipre black and white', 1),
 ('gar', 1),
 ('mojarra', 1),
 ('gray fish', 1),
 ('sand perch', 1),
 ('red snapper', 1),
 ('la chipa', 1),
 ('redfish', 1),
 ('tripletail', 1),
 ('puffer fish', 1),
 ('not sure', 1),
 ('smaller fish', 1)]

Here we've seen our verbal and our recorded responses in the survey! Amidst the recorded responses, people often selected various types of catfish, primarily channel catfish. In the verbal responses red and black drum were most commonly mentioned.

### Fishing Locations
<br> NOTE: We have a heat map that qualtrics created from a map in our survey that users could click on! I won't present that data here b/c it's well presented in qualtrics, our write-up, and our presentation. But here's the list of verbal responses for places people fish that they couldn't find on the map:

In [15]:
print_freeform_columns(['FREE FORM fishing locations'])

>>> fishing locations (verbal responses): 
Beaches like wrightsville. Kerr. Carolina. 
Fort Fischer. Wrightsville Beach in Bridge. Red man creek by Wilmington 
Snows cut 
Snows cut, wrightsville beach, NC wildlife ramp under bridge 
River road st Park and up and down stream
Cure beach
snows cut, Carolina beach inlet
Resize map
Princess Place, Carolina beach st park, snows cut 
Belville 
Curry beach
I Intercostal waters. Mouth of river
Roanoke rapids
Solomon towers, chamber of commerce, love grove
Snows cut
Black river
Eagle island bait shop on 421. Talk to people there
Beaches and piers


### Quantity of Fish Consumed

In [18]:
df_completed['portions in a meal'].value_counts().sort_index()

About 1 portion         17
About 2 portions        17
About 3 portions         6
About 4 portions         2
About 5 portions         2
More than 5 portions     1
Name: portions in a meal, dtype: int64

In [19]:
df_completed['meals per month'].value_counts().sort_index()

1-3 meals            34
4-6 meals             6
7-9 meals             1
More than 9 meals     4
Name: meals per month, dtype: int64

The above indicates that 1 to 2 portions in a meal are most common, and that overwhelmingly our respondents report eating just one to three meals per month.

### Fish Sharing Behaviors
____
<font color = 'red'> NOTE: In the original survey we included an option for people to say "Both: I eat it myself and share with others." We've since realized this was unnecessary b/c we ask at the beginning of the survey if people eat the fish themselves, so if a respondent is at this point in the survey then we know that they eat fish! So, in this dataset I replaced all the "Both" options with "Yes"

In [20]:
df_completed['shares fish with others'].value_counts()

Yes    41
No      4
Name: shares fish with others, dtype: int64

In [21]:
df_completed['number of people <15 '].value_counts().sort_index()

0            25
1 to 3       12
4 to 6        2
6 or more     1
Name: number of people <15 , dtype: int64

In [22]:
df_completed['number of fem 15-44'].value_counts().sort_index()

0            13
1 to 3       24
4 to 6        1
6 or more     3
Name: number of fem 15-44, dtype: int64

From this it is clear that sharing is very common. It mostly occurs with women of childbearing age, but a substantial number of respondents also report sharing with children.

## More Detailed Results

### What do People do With Fish Other Than Eating?

In [16]:
other_resp_dict = aggregate_mult_choice('fish use other than eating')
other_resp_dict

{'catch and release': 29, 'other': 8}

And let's take a look at some of the verbal responses we noted:

In [17]:
print_freeform_columns(['FREE FORM other use'])

>>> other use (verbal responses): 
Give away too
Share with friends and family 
Share an old couple
Share fish with neighbors 
Compartir
Cut up shad for bait
Share it, use for bait
Give away free. Neighbors family friends. 


This question shows that catch and release is very common, and the verbal responses indicate a high degree of sharing fish out in the community.

### How are People Preparing and Cooking the Fish?

In [23]:
cooking_methods = aggregate_mult_choice('cooking methods')
cooking_methods

{'fried': 38,
 'baked': 17,
 'grilled': 15,
 'other (explain):': 10,
 'steamed': 3,
 'smoked': 4,
 'stewed': 2,
 'boiled': 2}

There were some interesting verbal responses to preparation methods. Here they are:

In [24]:
print_freeform_columns(['FREE FORM cooking'])

>>> cooking (verbal responses): 
Blackened 
Sopa
Blackened 
Almost entirely catch and release
Ceviche con limon, parrillar con limon, estofado 
Cast iron skillet
Broil
Flour and cornmeal in hot grease
Grill with wood chips
Already spiced with genes


^^^ Note that the last verbal response has a typo. This person said 'Already spiced with genex'.

In [25]:
preparation_methods = aggregate_mult_choice('preparation methods')
sorted(preparation_methods.items(), key = lambda x: x[1], reverse = True)  # This prints the dictionary in order

[('remove scales', 32),
 ('remove fish organs', 30),
 ('fillet (only meat with no fat skin or bones)', 29),
 ('remove head', 27),
 ('remove skin', 18),
 ('remove fat', 8),
 ('other (explain):', 6)]

There were also some verbal responses about preparation methods, lets check those out:

In [26]:
print_freeform_columns(['FREE FORM preparation'])

>>> preparation (verbal responses): 
Limon para evitar bacteria, 
Butterfly it and grill it with onions
Every once in a while pan fry whole fish no head
A veces piel y cabeza
Stuff with shrimp, crab, red pepper, yellow 
Soak in sea salt water 24 hours. Takes fish taste out


These results show the popular cooking methods (primarily frying) and the most common means of preparation as well as some interesting verbal explanations

### Are Fishing Advisories Visible and Easy to Understand?

In [27]:
df_completed['seen fish advisories'].value_counts()

Yes    30
No     15
Name: seen fish advisories, dtype: int64

### Pollution Concerns
We asked people whether or not they were concerned about pollution in the river, and got some insightful follow up verbal responses!

In [28]:
df_completed['pollution concerns'].value_counts()

Yes    37
No      7
Name: pollution concerns, dtype: int64

In [29]:
print_freeform_columns(['FREE FORM pollution concerns'])

>>> pollution concerns (verbal responses): 
People need to stop dumping in the river
Mercury , sewage 
Checks but nothing really. Checks for worms
Fish population are low, hard to reproduce with limited fish. Health effects from chemicals. 
La basura contamina el río. Every time I see trash I pick it up
Cancer and health issues. Gen X , coal ash, dead fish kills
sewage. More salt now cuz its better
unsure whats in it...so thats why. Something is wrong with cfr
Down the river sewage discharge from norchase. Just knowing what’s on the river. A chemical plant producing chromium. Hog farms and chicken farms
Plastic and waste goes in.
pollution in river gets him worried
Littering at fishing spots and water 
Don’t know what’s in water. Gasoline, genex
Que los peces están muriendo
Bacteria, salmonela, but ceviche helps
Mercury and lead . Fish eating trash and parasites. 
Contamination. Dumping in water. Not just trash
Sewage outlet at wrightsville beach. And genex
Dead fish, plastic trash
Con

Indicates that people are overwhelmingly concerned about pollution in the river and in the fish. The verbal responses shed light on the many forms that these concerns take!

### Sources of Information

#### Where do people get information on which fish are safe to eat?

In [30]:
sources_of_info = aggregate_mult_choice('sources of information')
sorted(sources_of_info.items(), key = lambda x: x[1], reverse = True)  # This prints the dictionary in order

[('news', 21),
 ('other', 17),
 ('internet', 15),
 ('other people that are fishing', 11),
 ('family or friends', 8),
 ('signs and posters', 5),
 ('government', 4),
 ('bait and fishing shops', 3),
 ('schools', 1),
 ('radio', 1)]

In [31]:
# And now for a look at the verbal responses that we got, clarifying these selections:
list_of_columns = ['FREE FORM bait and tackle shops', 'FREE FORM news sources', 'FREE FORM internet sources', 
                   'FREE FORM government sources', 'FREE FORM sign sources', 'FREE FORM other sources']
print_freeform_columns(list_of_columns)

>>> bait and tackle shops (verbal responses): 
tex's tackle
all I visit
>>> news sources (verbal responses): 
Channel 6
Local channel 26.1. Local tv not cable
Carolina outdoor (tv program)
Morningstar, google for waste entering smith creek
Tv local
wilmington newspaper
local (wect, wway)
>>> internet sources (verbal responses): 
Tide table website
Science 
NC wildlife resources commission
>>> government sources (verbal responses): 
Nc dept of wildlife
state
marine fisheries, noaa, samfc
north carolina public health and DEQ website, EPA
>>> sign sources (verbal responses): 
Signs that say don’t leave trash
boat ramps
all posted signage
>>> other sources (verbal responses): 
Wildlife officer
Fish Rules
Personal knowledge. 
Commercial fishermen that are out there alll the time 
license vendors
Work
I see it
Fisherman’s post. Distributed in bait shops and restaurants in the summer
Magazines
fishing magazine available at store that sells licenses. Also people buy licenses online
The interne

Now a look at the verbal responses for when we asked, "what would be the best way of sharing such information?"

In [32]:
print_freeform_columns(['FREE FORM best sources'])

>>> best sources (verbal responses): 
News and bait shops
Seminar with local fishermen. They’d be happy to attend if it’s about water quality or fish. County arboretum. Or county natural resources dept. 
Tackle shop near Wrightsville beach, tackle shops
Boat landings. 
Radio in Spanish, hablar con personas
Word of mouth 
Televisión y radio. Radio la grande, El patrón
tackle shops. Island bait and tackle. 
Advisories and information when you get licens
Through talking to other fishers
Tackle shops
No idea
Social media. Bait shops. Word of mouth. Academy. 
Bait shops. 
Word of mouth and internet
No se
sea view
Word. Mouth
Post at dock (behind glass)
Johnny Mercer pier, curry beach pier, state, signs or wallet, 
Online is good
Bait shop
Bait shops
Facebook follow cape fear river watch
YouTube, fishbrain
Talk to people
Personal experience to get info. 
Que tengan cuidado y laven bien. Radio. Videos.
If we can get signs, that’s best
Bait shops. Walmart. Rural people drawn in
Fisherman’s.com

#### Where do people get trusted health information?
<br> Also regarding sources of information, we asked where people get trusted health information:

In [33]:
health_info = aggregate_mult_choice('health information sources')
sorted(health_info.items(), key = lambda x: x[1], reverse = True)  # This prints the dictionary in order

[('health clinic', 13),
 ('internet', 12),
 ('other', 8),
 ('family/friends', 4),
 ('church', 1)]

In [34]:
freeform_health_sources = ['FREE FORM internet health sources', 'FREE FORM other health sources']
print_freeform_columns(freeform_health_sources)

>>> internet health sources (verbal responses): 
Fishing sites
Fisherman’s guide
Epa, fisheries division, state of nc
Pub med
Government Websites - I used EPA website a lot but there is not much information there anymore.  I use the NC public health website now most often
>>> other health sources (verbal responses): 
Dept of wildlife resource.  
news. Like genx
Work and learning chrmicals.
Stores or someone who works at international paper
Healthdept
Postings on dock, fishing magazine cautions
News
physician


We see that people refer to many sources of information! The follow up on selections in the verbal responses shows some reliance on personal knowledge and also the specific news sources, websites and government agencies that people learn from. Finally, the verbal responses we recorded about best sources of information frequently mention tackle shops, signage, and many other creative information sources that this project might leverage.

#### Fish Advisories: Are they useful, why or why don't they affect people's decisions?

In [35]:
columns = ['seen fish advisories', 'fish advisories easy to understand', 'fish advisories affect decision']
for header in columns:
    fish_advisory_resp = aggregate_mult_choice(header)
    print(header + ': ')
    print(fish_advisory_resp)

seen fish advisories: 
{'yes': 30, 'no': 15}
fish advisories easy to understand: 
{'yes': 27, 'no': 2}
fish advisories affect decision: 
{'yes': 15, 'no': 13}


We asked for follow-up information on whether or not fish advisories affected individual choices ot eat fish:

In [36]:
# And now for a look at the verbal responses that we got, clarifying these selections:
list_of_columns = ['FREE FORM affected decision', 'FREE FORM did not affect decision']
print_freeform_columns(list_of_columns)

>>> affected decision (verbal responses): 
Stay away from high mercury fish
A little bit. Health reasons, good to know what’s In my body 
Si hay una señal que dice no comer, no lo haria
Not very specific but they work
Taking no chances. 
If posted yes
Coming from New York you only want to eat one fish a month. Still learning down here. Thinks it’s pretty bad here 
Somewhat. In a way cuz she doesn’t eat them every day. 
Because of the risk
Yes.  I always follow those guidelines
>>> did not affect decision (verbal responses): 
doesn't care. Worry about Kids."Gonna die anyway"
Do my thing
Picky eater 
Only on tv did they see that there’s a type of fish not to eat
Not really. Taste matters. If taste bad then not eat 
No beacuse he just cooks it well
No because going after particular fish that the signs don’t talk about
Don’t read a whole lot of it
Catfish there’s no limit on what you can eat
Brother does fishing
Different fish


These responses suggest that people see and mostly understand fishing advisories. Responses are mixed with regards to whether or not these advisories influence decisions. For those that were influenced by fishing advisories, the reason was often to err on the side of caution or just trusting that if a sign is posted it is in their best interest to follow its advice. For those that were influenced by signage, responses exhibited self-reliance or instances where the fish mentioned on signs don't correspond to what they catch.

### Feedback on Informational Materials:

#### Wallet Card
First we had a multiple choice question about what people took away from the card:

In [37]:
wallet_card_message = aggregate_mult_choice('wallet card takeaway')
sorted(wallet_card_message.items(), key = lambda x: x[1], reverse = True)  # This prints the dictionary in order

[('other', 24),
 ('some fish may have more contaminants than others', 22),
 ('some fish are safer to eat than others', 18),
 ('pregnant women and women between age 15-44 should not eat fish that may have mercury',
  10)]

People followed up on their 'other' takeaways verbally. Here are those responses:

In [38]:
print_freeform_columns(['FREE FORM other takeaway'])

>>> other takeaway (verbal responses): 
Check what your eating 
Por algunos pescados hay que leer avisos antes de comer
Stop and check for contamination, where? 
fish variety
website was noticeable. 
Catfish are bottom feeders
Shrimps not a fish. Humans are the cause of the contamination 
motto. Two different messages . To eat and to catch
Got Tv news about fish contaminants 
I would think they’re all polluted the same
Striped mullet run
Stop and check
Interesting that striped bass is on there. Wouldn’t think high levels of mercury, disturbing to see blue crab. 
Don’t eat too many of the same 
Good public advisory 
Saltwater fish safe to eat except catch fish. Catfish everyone knows have lots of mercury
The safest ones are saltymp
Freshwater. Comment on blue crabs. 
Can’t read
Catfish and blue crab are scavengers
Where I can call
We have a problem
Interesting


We also asked whether or not the wallet card seemed useful:

In [39]:
wallet_card_useful = aggregate_mult_choice('wallet card useful')
sorted(wallet_card_useful.items(), key = lambda x: x[1], reverse = True)  # This prints the dictionary in order

[('yes', 31), ('no', 10)]

And finally we collected some free form responses on why or why not the respondent found it useful, and what we could do to improve it:

In [40]:
list_of_columns = ['FREE FORM wallet card useful', 'FREE FORM wallet card not useful', 'FREE FORM improve wallet card']
print_freeform_columns(list_of_columns)

>>> wallet card useful (verbal responses): 
Especially tourists
Easy to remember content
Yes. Add this to license purchases. 
Especially people that don’t know about mercury
For people not from around here this would be useful.
If they don’t know. Experienced angler should know what to eat. 
People with kids probs will. 
Maybe.   May check it 
fits in wallet
If they are interested in food safety they would check the cards.  If they are not interested in water quality and its effects then they aren't going to have this wallet card.
If those are with chemicals and contamination is probably
I have one for ticks so I would use this one too.
>>> wallet card not useful (verbal responses): 
Probably not fisherman make mind up after catching fish and examining
When you get your permit you get all that info. Also he doesn’t know the name ps of all the fish
Nice but ppl do what they want to do. If he had family he'd be concerned. 
not sure
If they’re gonna catch fish they’ll eat them
Upon gettin

#### Refrigerator Magnet
Only the people who reported not knowing how to fillet were shown the magnet, so we only got a few responses since the overwhelming majority of respondents reported knowing how to fillet

In [41]:
magnet_message = aggregate_mult_choice('magnet main message')
sorted(magnet_message.items(), key = lambda x: x[1], reverse = True)  # This prints the dictionary in orderT

[('fish contaminants are stored in their fat', 2),
 ('other', 1),
 ('it is healthier to fillet a fish before eating it', 1)]

There were a couple follow up responses on the magnets, just saying its useful and that filleting seems to be better for their health but not worth printing here. Also, our two responses from the WIC clinics were providing this feedback on the brochures, but didn't have much feedback other than the need to translate!

### Likelihood, barriers, and motivations for different behaviors

In [42]:
columns = ['would eat fewer polluted fish', 'would eat more lower pollution fish', 'would fillet fish', 
           'would buy fish from market', 'would grill or bake rather than fry', 'would stop eating fish']
for header in columns:
    behavior_likelihood = aggregate_mult_choice(header)
    print('>>> ' + header + ': ')
    print(sorted(behavior_likelihood.items(), key = lambda x: x[1], reverse = True))

>>> would eat fewer polluted fish: 
[('probably will do', 12), ('definitely will do', 11), ('probably not do', 9), ('definitely not do', 5), ('neutral', 4)]
>>> would eat more lower pollution fish: 
[('probably will do', 19), ('definitely will do', 12), ('probably not do', 7), ('definitely not do', 2), ('neutral', 1)]
>>> would fillet fish: 
[('definitely will do', 17), ('probably will do', 16), ('neutral', 5), ('probably not do', 3)]
>>> would buy fish from market: 
[('probably will do', 16), ('definitely not do', 10), ('probably not do', 6), ('neutral', 5), ('definitely will do', 4)]
>>> would grill or bake rather than fry: 
[('probably will do', 15), ('definitely will do', 11), ('probably not do', 10), ('neutral', 3), ('definitely not do', 2)]
>>> would stop eating fish: 
[('definitely not do', 19), ('probably will do', 8), ('probably not do', 7), ('neutral', 3), ('definitely will do', 1)]


Please note, this output can be seen in a much more visually appealing format in our presentation! But this does show how we aggregated the responses. For the above, I print out the most commonly selected choice first, next to the number of times it was chosen.

We got a ton of clarification on people's choices. Here I'll list what we recorded from people's free form responses by which behavior they were responding to:

In [43]:
list_of_columns = ['FREE FORM other risk reduction option', 'FREE FORM eat less polluted fish', 'FREE FORM eat more fish with lower pollution', 'FREE FORM fillet fish', 'FREE FORM grocery store fish', 'FREE FORM other cooking methods', 'FREE FORM stop eating fish']
print_freeform_columns(list_of_columns)

>>> other risk reduction option (verbal responses): 
Deep fry the fish to kill bacteria and get rid of contaminant 
Limit the fish I eat. From there.  
>>> eat less polluted fish (verbal responses): 
I like it fresh
Don’t eat catfish or bass
Does not
Fish is fish, 
People will eat what they are going to eat
Recent immigrants don’t yet understand regulations or what’s safe vs, not. Also maybe don’t care
People have preferences for fish
People eat what they catch
Catfish are what people want to catch
He likes them
People like to eat these
i don't like eating those types of fish
>>> eat more fish with lower pollution (verbal responses): 
likes shrimp
Same as above
Kingfish looked good. People don’t like shad or spot or bluegill. Shrimp is alright, so are specks.
People eat what they catch
Some people come to eat what they want but will follow signs
If they’re big enough. Otherwise bait. These are all bait!
May eat these
i like to eat speckled trout
>>> fillet fish (verbal responses): 
Alg

### Final Comments
That's it for the official data. Here were all the last things people had to say!

In [44]:
print_freeform_columns(['FREE FORM last comments'])

>>> last comments (verbal responses): 
Nice meeting ya
Concerned about sewage in water. 
Tratar de tener limpio los sitios que uno va a pescar para no tener problema con nadie. Con la ley. Botar basura
When is more research gonna be done? They know these companies are here, and they should be the ones to do it. 
would like to know more about the whole deal with CFR
Appreciate us doing this and working on this. Anything we can do to reduce pollution.
You can see the pollution especially plástic and it’s impact on wildlife
In army
Good luck
Compartir más 
Me encanto la pesca
Fishing is good for you. Relaxing and stress free. 
Going further up the river
What we’re doing is important for the environment and for the people. People only learn not to fish if people die. We need science to forewarn us.
Fillet on the card. Put it at bait shops and grocery stores. Also around Southport . Keep talking to people
Glad we’re doing it
Nope
Good work
Glad there’s people out here working in the environ